In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
from scipy.integrate import simps
from time import time
from scipy.special import erf

In [ ]:
pm_data_LAE = np.load('npy/pm_eLAE_1deg.npy')
f_name = 'Source_cats/Source_cat_LAE_1deg.npy'
mock_LAE = np.load(f_name, allow_pickle=True).item()

pm_data_OII = np.load('npy/pm_eOII_1deg.npy')
f_name = 'Source_cats/Source_cat_OII_1deg.npy'
mock_OII = np.load(f_name, allow_pickle=True).item()

pm_data = np.zeros((pm_data_LAE.shape[0], pm_data_LAE.shape[1] + pm_data_OII.shape[1]))
pm_data[:, :pm_data_LAE.shape[1]] = pm_data_LAE
pm_data[:, pm_data_LAE.shape[1]:] = pm_data_OII

mock = {
    'SEDs': np.zeros((len(mock_LAE['SEDs']) + len(mock_OII['SEDs']), mock_LAE['SEDs'].shape[1])),
    'SEDs_no_line': np.zeros((len(mock_LAE['SEDs']) + len(mock_OII['SEDs']), mock_LAE['SEDs'].shape[1])),
    'w_Arr': mock_LAE['w_Arr']
}

mock['SEDs'][:len(mock_LAE['SEDs'])] = mock_LAE['SEDs']
mock['SEDs'][len(mock_LAE['SEDs']):] = mock_OII['SEDs']
mock['SEDs_no_line'][:len(mock_LAE['SEDs_no_line'])] = mock_LAE['SEDs_no_line']
mock['SEDs_no_line'][len(mock_LAE['SEDs_no_line']):] = mock_OII['SEDs_no_line']

errors = np.load('npy/errors5Sigma.npy')[:,1]

pm_data.shape

In [ ]:
filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

In [ ]:
bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
# The model function
def model_f(x, m, b):
    return m*x + b 

In [ ]:
nb_c = 14
N_nb_stack = 2
ew0min = 50
nb_fwhm_Arr = [nb_fwhm(tcurves, idx, True) for idx in np.arange(len(filters_tags))]
err = (np.ones(pm_data.shape).T * errors).T
cont_stack, cont_err_stack = stack_estimation(
    pm_data, err, nb_c, N_nb_stack, w_central, nb_fwhm_Arr, ew0min
)

In [ ]:
# Synthetic photometry of the continuum using the central NB
w_t = tcurves['w'][nb_c]
t = tcurves['t'][nb_c]
w_Arr = mock['w_Arr']
real_cont_value = np.zeros(len(mock['SEDs']))
i = 0
for m_SED in mock['SEDs_no_line']:
    t_interp = np.interp(w_Arr, w_t, t)
    real_cont_value[i] = simps(t_interp * m_SED * w_Arr, w_Arr) / simps(t_interp * w_Arr, w_Arr)
    i += 1

In [ ]:
plt.close('all')
for mock_n in np.random.randint(0, pm_data.shape[1], 20):
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots(figsize = (12, 8))
    ax.plot( mock['w_Arr'], mock['SEDs'][mock_n], c='orange', lw=2, zorder=-1)
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], c='gray', fmt='.')
    ax.scatter(w_central[nb_c], real_cont_value[mock_n], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4],
                xerr=bb_fwhm[-4]/2, yerr=errors[-4],
                fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], yerr=errors[-3],
                xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], yerr=errors[-2],
                xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], yerr=errors[-1],
                xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    ax.errorbar(w_central[nb_c]+10, cont_stack[mock_n], yerr=cont_err_stack[mock_n],
                c='violet', marker='^', markersize=9,
                capsize=4, label='Stack NBs', elinewidth=2, capthick=2)
    
    plt.legend()

    plt.show()

In [ ]:
for N in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    cont_stack_N, cont_err_stack_N = stack_estimation(
        pm_data, err, nb_c, N, w_central, nb_fwhm_Arr, ew0min
    )
    mean = np.average(np.abs((cont_stack_N - real_cont_value) / real_cont_value))
    std = np.std(np.abs((cont_stack_N - real_cont_value) / real_cont_value))
    print('N = {} | mean, std: {}, {}'.format(N, mean, std))

In [ ]:
N_sources = pm_data.shape[1]
N_sources

In [ ]:
for N in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    cont_stack_N, cont_err_stack_N = stack_estimation(
        pm_data, err, nb_c, N, w_central, nb_fwhm_Arr, ew0min
    )
    
    bias = np.average((cont_stack_N - real_cont_value) / real_cont_value)

    #### ERROR TESTING ####

    N_points = 20
    sigma_min = 0.5
    sigma_max = 3

    sigma_Arr = np.linspace(sigma_min, sigma_max, N_points)
    sigma_hist = np.zeros(N_points)
    sigma_hist_debiased = np.zeros(N_points)

    for i in range(N_points):
        sigma_hist[i] = len(
            np.where(
                np.abs(cont_stack_N - real_cont_value) < sigma_Arr[i] * cont_err_stack
            )[0]
        )
        sigma_hist_debiased[i] = len(
            np.where(
                np.abs(
                    (cont_stack_N - real_cont_value) - bias*real_cont_value
                ) < sigma_Arr[i] * cont_err_stack
            )[0]
        )


    # Plot the error test
    fig, ax =  plt.subplots(figsize = (10,7))

    ax.plot(sigma_Arr, sigma_hist/N_sources, marker='.', linestyle='none', label='Raw')
    ax.plot(sigma_Arr, sigma_hist_debiased/N_sources, marker='x', linestyle='none', label='Debiased')
    ax.plot(sigma_Arr, erf(sigma_Arr / 2**0.5), linestyle='dashed', label='erf$(\sigma/\sqrt{2})$')
    

    ax.set_ylabel('N / N_sources', fontsize=15)
    ax.set_xlabel('$\sigma$', fontsize=15)

    ax.set_title('N = {}'.format(N), fontsize=15)

    ax.legend(fontsize=14)

    plt.show()